# Feature Engineering

The next step is to create features from the raw text so we can train the machine learning models. The steps followed are:

1. **Text Cleaning and Preparation**: cleaning of special characters, downcasing, punctuation signs. possessive pronouns and stop words removal and lemmatization. 
2. **Label coding**: creation of a dictionary to map each category to a code.
3. **Train-test split**: to test the models on unseen data.
4. **Text representation**: use of TF-IDF scores to represent text.

In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

First of all we'll load the dataset:

In [2]:
path_df = "../02. Exploratory Data Analysis/News_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [3]:
df.head()

,File_Name,Content,Category,Complete_Filename,id,News_length
0,003.txt,"Safety Dividend \n\n\nSince then, Waymo has i...",opportunities,003.txt-opportunities,1,277
1,002.txt,"Changing Oil Demand \n\n\nIn 2009, Google sta...",opportunities,002.txt-opportunities,1,294
2,001.txt,Infrastructure Transformation \n\n\nImagine g...,opportunities,001.txt-opportunities,1,410
3,011.txt,"Promised solutions, emerging problems\n\nMuch ...",opportunities,011.txt-opportunities,1,2434
4,012.txt,"3 trends are driving investment\n\nSo, why do ...",opportunities,012.txt-opportunities,1,1978


And visualize one sample news content:

In [4]:
df.loc[1]['Content']

" Changing Oil Demand \n\n\nIn 2009, Google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes. In 2016, Waymo, an autonomous driving technology company, became a subsidiary of Alphabet, and Google's self-driving project became Waymo.\n"

## 1. Text cleaning and preparation

### 1.1. Special character cleaning

We can see the following special characters:

* ``\r``
* ``\n``
* ``\`` before possessive pronouns (`government's = government\'s`)
* ``\`` before possessive pronouns 2 (`Yukos'` = `Yukos\'`)
* ``"`` when quoting text

In [5]:
# \r and \n
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

Regarding 3rd and 4th bullet, although it seems there is a special character, it won't affect us since it is not a *real* character:

In [6]:
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [7]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

### 1.2. Upcase/downcase

We'll downcase the texts because we want, for example, `Football` and `football` to be the same word.

In [8]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

### 1.3. Punctuation signs

Punctuation signs won't have any predicting power, so we'll just get rid of them.

In [9]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

/var/folders/k5/nfk81ydj68d5jbhcvqbb71b40000gn/T/ipykernel_13904/1781969415.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


By doing this we are messing up with some numbers, but it's no problem since we aren't expecting any predicting power from them.

### 1.4. Possessive pronouns

We'll also remove possessive pronoun terminations:

In [10]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

### 1.5. Stemming and Lemmatization

Since stemming can produce output words that don't exist, we'll only use a lemmatization process at this moment. Lemmatization takes into consideration the morphological analysis of the words and returns words that do exist, so it will be more useful for us.

In [11]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to /Users/aashish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/aashish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In order to lemmatize, we have to iterate through every word:

In [13]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [14]:
df['Content_Parsed_5'] = lemmatized_text_list

Although lemmatization doesn't work perfectly in all cases (as can be seen in the example below), it can be useful.

### 1.6. Stop words

In [15]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aashish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [17]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

To remove the stop words, we'll handle a regular expression only detecting whole words, as seen in the following example:

In [18]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

We can now loop through all the stop words:

In [19]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

/var/folders/k5/nfk81ydj68d5jbhcvqbb71b40000gn/T/ipykernel_13904/3814005232.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


We have some dobule/triple spaces between words because of the replacements. However, it's not a problem because we'll tokenize by the spaces later.

As an example, we'll show an original news article and its modifications throughout the process:

In [20]:
df.loc[5]['Content']

'The future of transport isn’t simple\n\nTransport systems are multifaceted. No one single app or technology will solve the challenges. And, as we are discovering, some of the purported solutions to problems might actually be making the situation worse. If the goal is to get people out of their cars (for better health and quality of life and a better environment), this will require more than a technology. Better infrastructure and public policies (including better integration of land uses and transport to reduce the need for transport) will be required – congestion pricing being one of those.That is not to say technological innovations are not welcome as part of the solution, but they are just that … “part” of the solution.'

1. Special character cleaning

In [21]:
df.loc[5]['Content_Parsed_1']

'The future of transport isn’t simple  Transport systems are multifaceted. No one single app or technology will solve the challenges. And, as we are discovering, some of the purported solutions to problems might actually be making the situation worse. If the goal is to get people out of their cars (for better health and quality of life and a better environment), this will require more than a technology. Better infrastructure and public policies (including better integration of land uses and transport to reduce the need for transport) will be required – congestion pricing being one of those.That is not to say technological innovations are not welcome as part of the solution, but they are just that … “part” of the solution.'

2. Upcase/downcase

In [22]:
df.loc[5]['Content_Parsed_2']

'the future of transport isn’t simple  transport systems are multifaceted. no one single app or technology will solve the challenges. and, as we are discovering, some of the purported solutions to problems might actually be making the situation worse. if the goal is to get people out of their cars (for better health and quality of life and a better environment), this will require more than a technology. better infrastructure and public policies (including better integration of land uses and transport to reduce the need for transport) will be required – congestion pricing being one of those.that is not to say technological innovations are not welcome as part of the solution, but they are just that … “part” of the solution.'

3. Punctuation signs

In [23]:
df.loc[5]['Content_Parsed_3']

'the future of transport isn’t simple  transport systems are multifaceted no one single app or technology will solve the challenges and as we are discovering some of the purported solutions to problems might actually be making the situation worse if the goal is to get people out of their cars (for better health and quality of life and a better environment) this will require more than a technology better infrastructure and public policies (including better integration of land uses and transport to reduce the need for transport) will be required – congestion pricing being one of thosethat is not to say technological innovations are not welcome as part of the solution but they are just that … “part” of the solution'

4. Possessive pronouns

In [24]:
df.loc[5]['Content_Parsed_4']

'the future of transport isn’t simple  transport systems are multifaceted no one single app or technology will solve the challenges and as we are discovering some of the purported solutions to problems might actually be making the situation worse if the goal is to get people out of their cars (for better health and quality of life and a better environment) this will require more than a technology better infrastructure and public policies (including better integration of land uses and transport to reduce the need for transport) will be required – congestion pricing being one of thosethat is not to say technological innovations are not welcome as part of the solution but they are just that … “part” of the solution'

5. Stemming and Lemmatization

In [25]:
df.loc[5]['Content_Parsed_5']

'the future of transport isn’t simple  transport systems be multifaceted no one single app or technology will solve the challenge and as we be discover some of the purport solutions to problems might actually be make the situation worse if the goal be to get people out of their cars (for better health and quality of life and a better environment) this will require more than a technology better infrastructure and public policies (including better integration of land use and transport to reduce the need for transport) will be require – congestion price be one of thosethat be not to say technological innovations be not welcome as part of the solution but they be just that … “part” of the solution'

6. Stop words

In [26]:
df.loc[5]['Content_Parsed_6']

' future  transport ’ simple  transport systems  multifaceted  one single app  technology  solve  challenge     discover    purport solutions  problems might actually  make  situation worse   goal   get people    cars ( better health  quality  life   better environment)   require    technology better infrastructure  public policies (including better integration  land use  transport  reduce  need  transport)   require – congestion price  one  thosethat    say technological innovations   welcome  part   solution      … “part”   solution'

Finally, we can delete the intermediate columns:

In [27]:
df.head(1)

,File_Name,Content,Category,Complete_Filename,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,003.txt,"Safety Dividend \n\n\nSince then, Waymo has i...",opportunities,003.txt-opportunities,1,277,"Safety Dividend Since then, Waymo has invited...","safety dividend since then, waymo has invited...",safety dividend since then waymo has invited ...,safety dividend since then waymo has invited ...,safety dividend since then waymo have invite ...,safety dividend since waymo invite public ...


In [28]:
list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [29]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed
0,003.txt,opportunities,003.txt-opportunities,"Safety Dividend \n\n\nSince then, Waymo has i...",safety dividend since waymo invite public ...
1,002.txt,opportunities,002.txt-opportunities,"Changing Oil Demand \n\n\nIn 2009, Google sta...",change oil demand 2009 google start self-dr...
2,001.txt,opportunities,001.txt-opportunities,Infrastructure Transformation \n\n\nImagine g...,infrastructure transformation imagine get c...
3,011.txt,opportunities,011.txt-opportunities,"Promised solutions, emerging problems\n\nMuch ...",promise solutions emerge problems much inte...
4,012.txt,opportunities,012.txt-opportunities,"3 trends are driving investment\n\nSo, why do ...",3 trend drive investment venture capitali...


**IMPORTANT:**

We need to remember that our model will gather the latest news articles from different newspapers every time we want. For that reason, we not only need to take into account the peculiarities of the training set articles, but also possible ones that are present in the gathered news articles.

For this reason, possible peculiarities have been studied in the *05. News Scraping* folder.

## 2. Label coding

We'll create a dictionary with the label codification:

In [30]:
category_codes = {
    'opportunities': 0,
    'challenges': 1
}

In [31]:
# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

In [32]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed,Category_Code
0,003.txt,opportunities,003.txt-opportunities,"Safety Dividend \n\n\nSince then, Waymo has i...",safety dividend since waymo invite public ...,0
1,002.txt,opportunities,002.txt-opportunities,"Changing Oil Demand \n\n\nIn 2009, Google sta...",change oil demand 2009 google start self-dr...,0
2,001.txt,opportunities,001.txt-opportunities,Infrastructure Transformation \n\n\nImagine g...,infrastructure transformation imagine get c...,0
3,011.txt,opportunities,011.txt-opportunities,"Promised solutions, emerging problems\n\nMuch ...",promise solutions emerge problems much inte...,0
4,012.txt,opportunities,012.txt-opportunities,"3 trends are driving investment\n\nSo, why do ...",3 trend drive investment venture capitali...,0


## 3. Train - test split

We'll set apart a test set to prove the quality of our models. We'll do Cross Validation in the train set in order to tune the hyperparameters and then test performance on the unseen data of the test set.

In [33]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

Since we don't have much observations (only 2.225), we'll choose a test set size of 15% of the full dataset.

## 4. Text representation

We have various options:

* Count Vectors as features
* TF-IDF Vectors as features
* Word Embeddings as features
* Text / NLP based features
* Topic Models as features

We'll use **TF-IDF Vectors** as features.

We have to define the different parameters:

* `ngram_range`: We want to consider both unigrams and bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document
    frequency strictly higher than the given threshold
* `min_df`: When building the vocabulary ignore terms that have a document
    frequency strictly lower than the given threshold.
* `max_features`: If not None, build a vocabulary that only consider the top
    max_features ordered by term frequency across the corpus.

See `TfidfVectorizer?` for further detail.

It needs to be mentioned that we are implicitly scaling our data when representing it as TF-IDF features with the argument `norm`.

In [37]:
# Parameter election
ngram_range = (1,2)
min_df = 5
max_df = 1.
max_features = 300

We have chosen these values as a first approximation. Since the models that we develop later have a very good predictive power, we'll stick to these values. But it has to be mentioned that different combinations could be tried in order to improve even more the accuracy of the models.

In [38]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(11, 12)
(3, 12)


Please note that we have fitted and then transformed the training set, but we have **only transformed** the **test set**.

We can use the Chi squared test in order to see what unigrams and bigrams are most correlated with each category:

In [41]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")


# 'challenges' category:
  . Most correlated unigrams:
. road
. cars
. safe
. object
. technology
  . Most correlated bigrams:
. 

# 'opportunities' category:
  . Most correlated unigrams:
. road
. cars
. safe
. object
. technology
  . Most correlated bigrams:
. 



As we can see, the unigrams correspond well to their category. However, bigrams do not. If we get the bigrams in our features:

In [42]:
bigrams

[]

We can see there are only six. This means the unigrams have more correlation with the category than the bigrams, and since we're restricting the number of features to the most representative 300, only a few bigrams are being considered.

Let's save the files we'll need in the next steps:

In [43]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)